In [1]:
import pandas as pd

full_df = pd.read_csv('../../Data/SMTO_2019/SMTO_2019_Complete_Input.csv')

school_codes = full_df['School'].unique().tolist()
uni_codes = full_df[full_df['School_Type'] == 'University']['School'].unique().tolist()

full_df.head()

,Liv_Arr,Children,Cars,Income,Home_Zone,School_Name,Campus,Work,Licence,Mode,...,Dist.SHH,Dist.SHT,Dist.MI,Dist.SC,Dist.SG,Dist.YK,Dist.YG,Dist.RY,Dist.OC,PD
0,Live with family/parents,0.0,0.0,NaN,3851.0,Centennial College,Progress Campus,NW,False,Transit,...,4.821538,16.18253,19.03615,54.90925,32.749820,40.58064,43.982800,33.227630,31.525830,36
1,Live with family/parents,0.0,1.0,NaN,181.0,Centennial College,Morningside Campus,NW,False,Transit,...,43.017220,51.23784,22.93553,22.04248,8.990107,9.21471,8.279897,10.301550,10.302330,4
2,Live with family/parents,0.0,2.0,I don't know,1039.0,Centennial College,Progress Campus,NW,False,Transit,...,71.352090,81.92638,54.53982,13.64408,35.730980,40.61661,29.418650,35.055550,36.449570,20
3,NaN,NaN,NaN,NaN,191.0,Centennial College,Progress Campus,NW,False,NaN,...,40.200800,48.42141,20.66308,21.85741,5.659206,11.77978,8.357460,6.970653,6.971434,4
4,Live with family/parents,1.0,1.0,NaN,600.0,Centennial College,Progress Campus,NW,False,NaN,...,61.537790,69.75841,41.28113,11.49956,25.838390,18.81172,15.841280,25.162960,26.824060,16


In [2]:
def income_var(x):
    if x == 'Less than $ 14,999':
        return 'Low'
    elif x == '$ 15,000 - 29,999':
        return 'Low'
    elif x == '$ 30,000 - 39,999':
        return 'Low'
    elif x == '$ 40,000 - 49,999':
        return 'Low'
    elif x == '$ 50,000 - 59,999':
        return 'Low'
    elif x == '$ 60,000 - 69,999':
        return 'Low'
    elif x == '$ 70,000 - 79,999':
        return 'Low'
    elif x == '$ 80,000 - 89,999':
        return 'Low'
    elif x == '$ 90,000 - 99,999':
        return 'High'
    elif x == '$ 90,000 - 99,999':
        return 'High'
    elif x == '$ 100,000 - 124,999':
        return 'High'
    elif x == '$ 125,000 - 149,999':
        return 'High'
    elif x == '$ 150,000 - 199,999':
        return 'High'
    elif x == '$ 200,000 +':
        return 'High'
    else:
        return 'Unknown'
    
full_df['Income'] = full_df['Income'].apply(lambda x: income_var(x))

In [3]:
full_df['Closest'] = full_df[['Dist.' + code for code in school_codes]].idxmin(axis = 1)
full_df['Closest_Type'] = full_df['Closest'].str.split('.').apply(lambda x: 1 if x[1] in uni_codes else 0)
full_df['Closest_College'] = (full_df['Closest_Type'] == 0) & (full_df['Family'] == False)
full_df['Closest_Uni'] = (full_df['Closest_Type'] == 1) & (full_df['Family'] == False)

# Filter df with our needed variables
variable_list = ['PD', 'Age', 'Cars', 'Family', 'Income', 'Closest_Type', 'Licence']
df = pd.concat((full_df['School_Type'], full_df[variable_list],), axis = 1)

# Make boolean variables
df['Income_High'] = (df['Income'] == 'High')*1
df['Income_Low'] = (df['Income'] == 'Low')*1
del df['Income']
df['Family'].fillna('Unknown', inplace = True)
df['Family_True'] = (df['Family'] == True)*1
df['Family_False'] = (df['Family'] == False)*1
del df['Family']
df['Cars2+'] = (df['Cars'] >= 2)*1
df['Cars1'] = (df['Cars'] == 1)*1
df['Cars0'] = (df['Cars'] == 0)*1
del df['Cars']
df['Licence_True'] = (df['Licence'] == 1)*1
df['Licence_False'] = (df['Licence'] == 0)*1
del df['Licence']

# Make X & Y the same length
df = df.dropna()

In [4]:
df.shape

(16902, 13)

In [5]:
df.head()

,School_Type,PD,Age,Closest_Type,Income_High,Income_Low,Family_True,Family_False,Cars2+,Cars1,Cars0,Licence_True,Licence_False
0,College,36,26.0,0,0,0,1,0,0,0,1,0,1
1,College,4,25.0,0,0,0,1,0,0,1,0,0,1
2,College,20,19.0,1,0,0,1,0,1,0,0,0,1
3,College,4,25.0,0,0,0,0,0,0,0,0,0,1
4,College,16,97.0,0,0,0,1,0,0,1,0,0,1


In [6]:
# Split 2019 data into initial training and testing
train_df = df.sample(frac=0.75, random_state=0)
test_df = df.drop(train_df.index)

In [7]:
train_df.to_csv('Training.csv')
test_df.to_csv('Testing_Set.csv')